# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [11]:
# Libraries
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admission_predict`
* table: `admission_predict`

In [15]:
# your code here
driver   = 'mysql+pymysql:'
user     = 'data-students'
password = 'iR0nH@cK-D4T4B4S3'
ip       = '34.65.10.136'
database = 'admission_predict'
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

pd.read_sql('SHOW TABLES;', engine)
predict = pd.read_sql('SELECT * FROM admission_predict', engine)
predict_original = pd.read_sql('SELECT * FROM admission_predict', engine)

Remove trailing spaces at the end of the column names if there are any.


In [26]:
# your code here
predict.rename(columns=lambda x: x.strip())
predict.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

Let's evaluate the dataset by looking at the `head` function.

In [31]:
# your code here
predict.head(5)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3,3.5,8,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.8
3,4,314,103,2,2,3,8.21,0,0.65
4,5,330,115,5,4.5,3,9.34,1,0.9


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [36]:
# your code here
predict.isnull().values.any()

False

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [53]:
# your code here
predict.set_index('Serial No.', inplace = True)
predict['Serial No.'] = predict.index
predict.head(5)

(385, 9)

Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [60]:
# your code here
GRE_Unique = len(predict['GRE Score']) == len(predict['GRE Score'].unique())
CGPA_Unique = len(predict['CGPA']) == len(predict['CGPA'].unique())

print(f'GRE is unique: {GRE_Unique}\nCGPA is unique: {CGPA_Unique}')

predict['GRE-CGPA'] = predict['GRE Score'].astype(str) + '-' + predict['CGPA'].astype(str)
predict.head(2)

len(predict['GRE-CGPA']) == len(predict.index)

GRE is unique: False
CGPA is unique: False


True

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

In [61]:
predict.set_index('GRE-CGPA', inplace = True)
predict.drop(columns= ['GRE Score', 'CGPA'], inplace = True)
predict.head(2)

,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,Serial No.
GRE-CGPA,,,,,,,
337-9.65,118,4,4.5,4.5,1,0.92,1
316-8,104,3,3,3.5,1,0.72,2


Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [62]:
# your code here
predict.reset_index(inplace= True)

In [63]:
predict.head(5)

,GRE-CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,Serial No.
0,337-9.65,118,4,4.5,4.5,1,0.92,1
1,316-8,104,3,3,3.5,1,0.72,2
2,322-8.67,110,3,3.5,2.5,1,0.8,3
3,314-8.21,103,2,2,3,0,0.65,4
4,330-9.34,115,5,4.5,3,1,0.9,5


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [69]:
predict = predict_original

In [88]:
# your code here
filter1 = predict[predict['CGPA'].astype(float) > 9]
len(filter1)

110

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [98]:
# your code here
filter2 = predict[(predict['CGPA'].astype(float) > 9) & (predict['SOP'].astype(float) < 3.5)]
filter2_mean = filter2['Chance of Admit'].astype(float).mean()
print(f'The mean chance for the group is {round(filter2_mean,3)}')

The mean chance for the group is 0.802


We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [104]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    col_mean = col.astype(float).mean()
    col_std = col.astype(float).std()
    result = col.astype(float).apply(lambda x: (x-col_mean)/col_std)
    return result
    
    # your code here
predict.head(0)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit


Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [106]:
# your code here
predict['CGPA_std'] = standardize(predict.CGPA)
predict['GRE_std'] = standardize(predict['GRE Score'])
predict['LOR_std'] = standardize(predict.LOR)
predict.head(5)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.747900,1.753381,1.191647
1,2,316,104,3,3,3.5,8,1,0.72,-0.991211,-0.063367,0.076740
2,3,322,110,3,3.5,2.5,8.67,1,0.8,0.121034,0.455704,-1.038166
3,4,314,103,2,2,3,8.21,0,0.65,-0.642597,-0.236391,-0.480713
4,5,330,115,5,4.5,3,9.34,1,0.9,1.233279,1.147798,-0.480713


We will generate the decision choice at random using the code below. Please run the cell.

In [107]:
# Libraries
from random import choices

In [109]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=predict.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [136]:
# your code here
predict['deciding_column'] = decision_choice
predict.head(10)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.747900,1.753381,1.191647,LOR_std
1,2,316,104,3,3,3.5,8,1,0.72,-0.991211,-0.063367,0.076740,LOR_std
2,3,322,110,3,3.5,2.5,8.67,1,0.8,0.121034,0.455704,-1.038166,LOR_std
3,4,314,103,2,2,3,8.21,0,0.65,-0.642597,-0.236391,-0.480713,CGPA_std
4,5,330,115,5,4.5,3,9.34,1,0.9,1.233279,1.147798,-0.480713,CGPA_std
5,6,321,109,3,3,4,8.2,1,0.75,-0.659198,0.369192,0.634193,CGPA_std
6,7,308,101,2,3,4,7.9,0,0.68,-1.157218,-0.755462,0.634193,GRE_std
7,8,302,102,1,2,1.5,8,0,0.5,-0.991211,-1.274533,-2.153072,CGPA_std
8,9,323,108,3,3.5,3,8.6,0,0.45,0.004829,0.542216,-0.480713,LOR_std
9,10,327,111,4,4,4.5,9,1,0.84,0.668856,0.888263,1.191647,LOR_std


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [137]:
# your code here
def checking (df):
    result = []
    for i in range(len(df.deciding_column)):
#         column_to_check = df.deciding_column[i]
#         print(column_to_check)
#         value_to_check = df[column_to_check][i]
#         print(value_to_check)
        if df[df.deciding_column[i]][i] > 0.8:
            result.append(1)
        else:
            result.append(0)
    return result

predict['decision'] = checking(predict)
predict.head(5)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.747900,1.753381,1.191647,LOR_std,1
1,2,316,104,3,3,3.5,8,1,0.72,-0.991211,-0.063367,0.076740,LOR_std,0
2,3,322,110,3,3.5,2.5,8.67,1,0.8,0.121034,0.455704,-1.038166,LOR_std,0
3,4,314,103,2,2,3,8.21,0,0.65,-0.642597,-0.236391,-0.480713,CGPA_std,0
4,5,330,115,5,4.5,3,9.34,1,0.9,1.233279,1.147798,-0.480713,CGPA_std,1


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [143]:
# your code here
print(len(predict[predict.decision == 1]), 'students will be accepted')

94 students will be accepted


# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [153]:
# your code here
predict.columns = predict.columns.str.replace(' ', '_')
predict.head(0)

,Serial_No.,GRE_Score,TOEFL_Score,University_Rating,SOP,LOR,CGPA,Research,Chance_of_Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [177]:
# your code here
def new_score(df):
    something = []
    for i in range(len(df.index)):
        if df.University_Rating[i] > 4:
            something.append(df.GRE_Score[i]+10)
        else:
            something.append(df.GRE_Score[i])
    return something
    
# # predict['adjusted_gre'] = predict.GRE_Score.apply(lambda predict[predict.University_Rating > 4][x]: x + 10)
predict['adjusted_gre'] = pd.cut(new_score(predict), 4,labels=["bad", "medium", "good", "very good"])
predict



,Serial_No.,GRE_Score,TOEFL_Score,University_Rating,SOP,LOR,CGPA,Research,Chance_of_Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision,adjusted_gre
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.747900,1.753381,1.191647,LOR_std,1,very good
1,2,316,104,3,3,3.5,8,1,0.72,-0.991211,-0.063367,0.076740,LOR_std,0,medium
2,3,322,110,3,3.5,2.5,8.67,1,0.8,0.121034,0.455704,-1.038166,LOR_std,0,good
3,4,314,103,2,2,3,8.21,0,0.65,-0.642597,-0.236391,-0.480713,CGPA_std,0,medium
4,5,330,115,5,4.5,3,9.34,1,0.9,1.233279,1.147798,-0.480713,CGPA_std,1,very good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.735259,0.628727,0.076740,GRE_std,0,good
381,382,325,107,3,3,3.5,9.11,1,0.84,0.851463,0.715239,0.076740,CGPA_std,1,good
382,383,330,116,4,5,4.5,9.45,1,0.91,1.415886,1.147798,1.191647,LOR_std,1,good
383,384,312,103,3,3.5,4,8.78,0,0.67,0.303641,-0.409414,0.634193,CGPA_std,0,medium
